In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "myPass123"
port = '42995'
database = 'AAC'
authDB = 'AAC'
shelter = AnimalShelter(username, password, port, database, authDB)
presses1 = 0
presses2 = 0 
presses3 = 0
presses4 = 0

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog'}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso salvare logo file
image_filename = 'GSLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),

    #Company logo centered at top of page with a link to the organization website in the image
    html.Center(
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))],
            href='https://www.snhu.edu'
        )
    ),
    #Header line with author name and organization
    html.Center(html.B(html.H1('Mitchell Lynds : SNHU CS 340 Dashboard'))),
    html.Hr(),
    
    html.Div(
        html.P(className='row',
             style={'display' : 'flex'},
                  children=[
                          #Display four buttons for different filter settings
                          html.Button(id='submit-button-one', n_clicks_timestamp=0, children='Water Rescue'),
                          html.Button(id='submit-button-two', n_clicks_timestamp=0, children='Mountain Rescue'),
                          html.Button(id='submit-button-three', n_clicks_timestamp=0, children='Disaster Rescue'),
                          html.Button(id='submit-button-four', n_clicks_timestamp=0, children='Reset')
                  ])
            ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        selected_columns=[],
        hidden_columns=[],
        page_action="native",
        page_current= 0,
        page_size= 10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              #pass button press timestamps 
              [Input('submit-button-one', 'n_clicks_timestamp'),Input('submit-button-two', 'n_clicks_timestamp'),
               Input('submit-button-three', 'n_clicks_timestamp'),Input('submit-button-four', 'n_clicks_timestamp')
              ])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries
    #base case for no clicks
    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) ==0 and int(bt4) ==0):
        #apply new query
        df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog'}))
        
    # use most recent timestamp of button clicks to determine filter type
    elif (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4)):
        #apply new query
        df = pd.DataFrame(list(shelter.read(
            {'breed': { '$in': ['Labrador Retriever Mix','Chesapeake Bay Retriever','Newfoundland']},
             'age_upon_outcome_in_weeks' : { '$gt' :  26, '$lt' : 156}, 
             'sex_upon_outcome' : 'Intact Female'})))
    elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
        #apply new query
        df = pd.DataFrame(list(shelter.read(
            {'breed': { '$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                'Siberian Husky', 'Rottweiler']},
             'age_upon_outcome_in_weeks' : { '$gt' :  26, '$lt' : 156}, 
             'sex_upon_outcome' : 'Intact Male'})))
    elif (int(bt3) > int(bt1) and int(bt3) > int(bt2) and int(bt3) > int(bt4)):
        #apply new query
        df = pd.DataFrame(list(shelter.read(
            {'breed': { '$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 
                                'Bloodhound', 'Rottweiler']},
             'age_upon_outcome_in_weeks' : { '$gt' :  26, '$lt' : 300}, 
             'sex_upon_outcome' : 'Intact Male'})))
    elif (int(bt4) > int(bt1) and int(bt4) > int(bt2) and int(bt4) > int(bt3)):
        #apply new query
        df = pd.DataFrame(list(shelter.read({'animal_type' : 'Dog'})))
    #generate table data
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    #return new data to table
    return (data, columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    #dataframe created from the data in the dash table
    df = pd.DataFrame.from_dict(viewData)
    #count number of occurrences of each breed in the filtered data
    percentages = df['breed'].value_counts().values
    #list of all breeds in the data set
    breeds = df['breed'].value_counts().index
    #return pie chart
    return [
        dcc.Graph(            
            figure = px.pie(df, values= percentages, names= breeds)
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#create dataframe from the data in the dash table
    dff = pd.DataFrame.from_dict(viewData)
    #Return table 
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
            dl.Marker(position=[(dff.iloc[0,13]),(dff.iloc[0,14])], children=[
                dl.Tooltip(dff.iloc[0,4]),
               dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app